In [1]:
import os
import sys
import glob
import gzip
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import cartopy.feature as cf
import cartopy.crs as ccrs

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
# product, time_reso = 'NOW','30min'
product, time_reso = 'NRT','1h'

In [3]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Brazil','GSMaP')

In [4]:
if product == 'NOW':
    time_total = 48
    dates_all = pd.date_range(
        start=f"2024-01-01 00:00:00",
        end=f"2024-12-31 23:59:00",
        freq="30min"
        ).strftime("%Y-%m-%d %H:%M:%S").tolist()
    description="GSMaP NOW 30 minutes data for South America"

elif product == 'NRT':
    time_total = 24
    dates_all = pd.date_range(
        start=f"2024-01-01 00:00:00",
        end=f"2024-12-31 23:59:00",
        freq="1h"
        ).strftime("%Y-%m-%d %H:%M:%S").tolist()
    description="GSMaP NRT 1 day data for South America"

print(f'Product: {product}')
print(f'Number of times: {len(dates_all)}')

Product: NRT
Number of times: 8784


In [5]:
sys.exit()

SystemExit: 

/home/arturo/anaconda3/envs/AXE/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Concat subdaily files in one daily file

In [5]:
# Create the dates for daily data as reference to read in each day directory
daily = pd.date_range(
        start=f"2024-01-01 00:00:00",
        end=f"2024-12-31 23:59:00",
        freq="1d"
        ).strftime("%Y-%m-%d").tolist()

In [6]:
print(f'Product: {product}')
print()
for nn in range(len(daily)):#len(daily)
    Tref = daily[nn]
    print(f'Processing: {Tref}')
    yy = Tref.split('-')[0]
    mm = Tref.split('-')[1]
    dd = Tref.split('-')[2]

    file_list = glob.glob(os.path.join(dir_base,product,'raw',yy,mm,dd,'*.gz'))
    file_list = sorted(file_list)
    # original_name = file_list[nn].split('/')[-1].split('_nrt.')[1].split('.dat.gz')[0]

    if product == 'NOW':
        dates_list = pd.date_range(
                start=f"2024-{mm}-{dd} 00:00:00",
                end=f"2024-{mm}-{dd} 23:30:00",
                freq="30min"
                ).strftime("%Y-%m-%d %H:%M:%S").tolist()
    else:
        dates_list = pd.date_range(
                start=f"2024-{mm}-{dd} 00:00:00",
                end=f"2024-{mm}-{dd} 23:30:00",
                freq="1h"
                ).strftime("%Y-%m-%d %H:%M:%S").tolist()

    pre_d = np.zeros([time_total,610,500])
    tim_d = []
    for pos in range(len(dates_list)):
        time_ref = pd.Timestamp(dates_list[pos]).strftime('%Y%m%d.%H%M')
        # print(f'Processing: {time_ref}')
        time_file = f'/media/arturo/Arturo/Data/Brazil/GSMaP/{product}/raw/{yy}/{mm}/{dd}/gsmap_{product.lower()}.{time_ref}.dat.gz'
        # if original_name != time_ref:
        #     print(f'ERROR: {original_name} and {time_ref}')
        try:
            with gzip.open(time_file, mode='rb') as handle:
                data_ = np.frombuffer(handle.read(), dtype=np.float32).reshape(610, 500)
                pre_d[pos,:,:] = data_ #np.round(data_,3)
                if product == 'NOW':
                    tim_d.append(pd.Timestamp(dates_list[pos]).strftime('%Y-%m-%d %H:%M'))
                else:
                    tim_d.append(pd.Timestamp(dates_list[pos]).strftime('%Y-%m-%d %H:%M'))
        except:
            print(f'File not found: {time_file}')
            pre_d[pos,:,:] = np.nan
            tim_d.append(pd.Timestamp(dates_list[pos]).strftime('%Y-%m-%d %H:%M'))

    # pre_d = pre_d.ravel()
    # pre_d = pre_d.astype(np.float32)

    # with gzip.open(os.path.join(dir_base,product,'raw_1dy',f'{product}_{yy}_{mm}_{dd}.dat.gz'), 'wb') as f:
    #     f.write(pre_d.tobytes())

Product: NRT

Processing: 2024-01-01
Processing: 2024-01-02
Processing: 2024-01-03
Processing: 2024-01-04
Processing: 2024-01-05
Processing: 2024-01-06
Processing: 2024-01-07
Processing: 2024-01-08
Processing: 2024-01-09
Processing: 2024-01-10
Processing: 2024-01-11
Processing: 2024-01-12
Processing: 2024-01-13
Processing: 2024-01-14
Processing: 2024-01-15
Processing: 2024-01-16
Processing: 2024-01-17
Processing: 2024-01-18
Processing: 2024-01-19
Processing: 2024-01-20
Processing: 2024-01-21
Processing: 2024-01-22
Processing: 2024-01-23
Processing: 2024-01-24
Processing: 2024-01-25
Processing: 2024-01-26
Processing: 2024-01-27
Processing: 2024-01-28
Processing: 2024-01-29
Processing: 2024-01-30
Processing: 2024-01-31
Processing: 2024-02-01
Processing: 2024-02-02
Processing: 2024-02-03
Processing: 2024-02-04
Processing: 2024-02-05
Processing: 2024-02-06
Processing: 2024-02-07
Processing: 2024-02-08
Processing: 2024-02-09
Processing: 2024-02-10
Processing: 2024-02-11
Processing: 2024-02-

In [ ]:
sys.exit()

## Concat all daily files in one yearly file

In [5]:
file_list = glob.glob(os.path.join(dir_base,product,'raw_1dy','*.gz'))
file_list = sorted(file_list)
file_len = len(file_list)
print(f'Files: {file_len}')

Files: 366


In [6]:
print(f'Product: {product}')

PRE_LIST = []
for nn in range(len(file_list)):
    with gzip.open(file_list[nn], mode='rb') as handle:
        data_ = np.frombuffer(handle.read(), dtype=np.float32).reshape(time_total, 610, 500)
        if product == 'NOW':
            PRE_LIST.append(data_/2)
        elif product == 'NRT':
            PRE_LIST.append(data_)

PRE_FINAL = np.concatenate(PRE_LIST, axis=0)
PRE_FINAL[PRE_FINAL < 0] = np.nan

Product: NRT


In [7]:
lat = np.arange(-55,6,0.1)[::-1]
lon = np.arange(-83,-33,0.1)
lon2d, lat2d = np.meshgrid(lon, lat)

In [8]:
# NOW 17568 times
# NRT 8784 times
PRE_FINAL.shape, len(dates_all), len(lat), len(lon)

((8784, 610, 500), 8784, 610, 500)

In [9]:
print(f'description: {description}')

PRE_xr = xr.Dataset(data_vars={"PRE": (("time","lat","lon"), PRE_FINAL)},
                    coords={'time': dates_all, 'lat': lat, 'lon': lon},
                    attrs=dict(description=description))

PRE_xr.PRE.attrs["units"] = "mm/h"
PRE_xr.PRE.attrs["long_name"] = "precipitation"
PRE_xr.PRE.attrs["origname"] = "precipitation"

PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lat.attrs["long_name"] = "Latitude"

PRE_xr.lon.attrs["units"] = "degrees_east"
PRE_xr.lon.attrs["long_name"] = "Longitude"

description: GSMaP NRT 1 day data for South America


In [10]:
yy_s = pd.to_datetime(dates_all[0]).year
mm_s = str(pd.to_datetime(dates_all[0]).month).zfill(2)
dd_s = str(pd.to_datetime(dates_all[0]).day).zfill(2)

yy_e = pd.to_datetime(dates_all[-1]).year
mm_e = str(pd.to_datetime(dates_all[-1]).month).zfill(2)
dd_e = str(pd.to_datetime(dates_all[-1]).day).zfill(2)

print(f'Start time: {dd_s}/{mm_s}/{yy_s}')
print(f'End time: {dd_e}/{mm_e}/{yy_e}')

Start time: 01/01/2024
End time: 31/12/2024


In [29]:
PRE_out = os.path.join(dir_base,product,f'GSMaP_{product}_SA_{time_reso}_{yy_s}_{mm_s}_{dd_s}_{yy_e}_{mm_e}_{dd_e}.nc')
print(f'Export PRE data to {PRE_out}')
PRE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/Arturo/Data/Brazil/GSMaP/NRT/GSMaP_NRT_SA_1h_2024_01_01_2024_12_31.nc
